In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from IPython.display import display, HTML

In [2]:
path = "data.xlsx"
df = pd.read_excel(path, sheet_name="1. District-level data")

In [3]:
for col in df.columns:
    print(repr(col))

'0a. Has committed ESBs?'
'1a. State'
'1b. Local Education Agency (LEA) or entity name'
'1c. LEA ID'
'1d. Street address 1'
'1e. Street address 2'
'1f. City'
'1g. State'
'1h. ZIP code'
'1i. Website'
'1j. Has LEA ID?'
'1k. LEA type (number)'
'1l. LEA type (name)'
'1m. Supervisory union LEA ID'
'1n. Locale (full number)'
'1o. Locale broad type (number)'
'1p. Locale broad type (name)'
'1q. Census Region'
'1r. Census Division'
'1s. Latitude'
'1t. Longitude '
'2a. Total number of buses'
'2b. Contractor used for some or all of buses?'
'3a. Number of ESBs committed '
'3b. Number of delivered or operating ESBs'
'3c. Number of ESBs awarded'
'3d. Number of ESBs ordered'
'3e. Number of ESBs delivered'
'3f. Number of ESBs operating'
'3g. Number of batches'
'3h. Size of batch 1'
'3h. Size of batch 2'
'3h. Size of batch 3'
'3h. Size of batch 4'
'3h. Size of batch 5'
'3i. Percent of fleet that is electric'
'3j. Government agency involved (non-funding)'
'3k. Utility/energy company involved 1'
'3k. Uti

In [4]:
cols = [
    '1b. Local Education Agency (LEA) or entity name',
    '1f. City',
    '1a. State',
    '2a. Total number of buses',
    '3a. Number of ESBs committed ',
    '4e. Percentage of students in district eligible for free or reduced price lunch',
    '5f. PM2.5 concentration',
    '4f. Median household income'
]
df = df[cols]

In [5]:
df.columns = [
    'district', 'city', 'state', 'total_buses', 'committed_esb',
    'free_lunch_pct', 'pm25', 'median_income'
]

In [6]:
df['free_lunch_pct'] = df['free_lunch_pct'] * 100

bakersfield_df = df[(df['city'].str.contains('Bakersfield', case=False, na=False)) & (df['state'].str.contains('California', case=False, na=False))]

print("Nombre de lignes Bakersfield:", len(bakersfield_df))
print(bakersfield_df[['district', 'city', 'state']])

Nombre de lignes Bakersfield: 18
                                 district         city       state
1256                      Kern County ROP  Bakersfield  CALIFORNIA
1257                        Kern High ROC  Bakersfield  CALIFORNIA
2432                     Bakersfield City  Bakersfield  CALIFORNIA
2443                 Beardsley Elementary  Bakersfield  CALIFORNIA
2487             Panama-Buena Vista Union  Bakersfield  CALIFORNIA
2664           General Shafter Elementary  Bakersfield  CALIFORNIA
2777                       Lakeside Union  Bakersfield  CALIFORNIA
2921                    Norris Elementary  Bakersfield  CALIFORNIA
3036   Rio Bravo-Greeley Union Elementary  Bakersfield  CALIFORNIA
3046            Rosedale Union Elementary  Bakersfield  CALIFORNIA
3154                  Standard Elementary  Bakersfield  CALIFORNIA
3435                    Edison Elementary  Bakersfield  CALIFORNIA
3961                   Fairfax Elementary  Bakersfield  CALIFORNIA
4835                 Fruitval

In [7]:
# === KPI 1: ESB Adoption Rate (%)
df = df.copy()
df['esb_adoption_rate'] = (df['committed_esb'] / df['total_buses']) * 100
bakersfield_df['esb_adoption_rate'] = (bakersfield_df['committed_esb'] / bakersfield_df['total_buses']) * 100

# === KPI 2: Air Pollution (PM2.5)
# Average air pollution level (micrograms per cubic meter) for Bakersfield
pm25 = bakersfield_df['pm25'].mean()

# === KPI 3: Median Household Income ($)
# Average median income across Bakersfield districts
income = bakersfield_df['median_income'].mean()

# === KPI 4: % of Students Eligible for Free/Reduced Lunch
# Social indicator measuring student economic vulnerability
free_lunch = bakersfield_df['free_lunch_pct'].mean()

# 🔹 Aggregate KPI 1 (Adoption Rate) across all Bakersfield districts
adoption = (bakersfield_df['committed_esb'].sum() / bakersfield_df['total_buses'].sum()) * 100

# === Display the 4 KPIs for Bakersfield, CA ===
print("\n=== KPI Results for Bakersfield (CA) ===")
print(f"ESB Adoption Rate: {adoption:.2f}%")                     # KPI 1
print(f"PM2.5 Level: {pm25:.2f} µg/m³")                          # KPI 2
print(f"Median Income: ${income:,.0f}")                          # KPI 3
print(f"Students eligible for free/reduced lunch: {free_lunch:.2f}%")  # KPI 4


=== KPI Results for Bakersfield (CA) ===
ESB Adoption Rate: 5.85%
PM2.5 Level: 9.56 µg/m³
Median Income: $70,796
Students eligible for free/reduced lunch: 71.68%


In [8]:
# === Create 4 KPI Cards (clean layout) ===
fig = make_subplots(rows=1, cols=4, specs=[[{"type": "indicator"}] * 4],
                    column_widths=[0.25, 0.25, 0.25, 0.25])

# KPI 1 – ESB Adoption Rate
fig.add_trace(go.Indicator(
    mode="number",
    value=adoption,
    number={'suffix': "%", 'font': {'size': 40}},
    title={"text": "<b>ESB Adoption</b><br>Rate"},
    domain={'x': [0, 0.25]}
), row=1, col=1)

# KPI 2 – Air Pollution (PM2.5)
fig.add_trace(go.Indicator(
    mode="number",
    value=pm25,
    number={'suffix': " µg/m³", 'font': {'size': 40}},
    title={"text": "<b>Air Pollution</b><br>(PM2.5)"},
    domain={'x': [0.25, 0.5]}
), row=1, col=2)

# KPI 3 – Median Income
fig.add_trace(go.Indicator(
    mode="number",
    value=income,
    number={'prefix': "$", 'font': {'size': 40}},
    title={"text": "<b>Median</b><br>Household Income"},
    domain={'x': [0.5, 0.75]}
), row=1, col=3)

# KPI 4 – Free/Reduced Lunch %
fig.add_trace(go.Indicator(
    mode="number",
    value=free_lunch,
    number={'suffix': "%", 'font': {'size': 40}},
    title={"text": "<b>Students</b><br>Free/Reduced Lunch"},
    domain={'x': [0.75, 1.0]}
), row=1, col=4)

# === Layout settings ===
fig.update_layout(
    title="<b>Bakersfield, CA – Key Performance Indicators</b>",
    template="plotly_white",
    height=200,     
    width=1000,     
    margin=dict(t=80, b=20, l=40, r=40)
)

fig.show()



In [9]:
# Compare ESB Adoption Rate (%) and PM2.5
state_df = df[df['state'].str.contains('California', case=False, na=False)]

state_mean = {
    'esb_adoption_rate': state_df['esb_adoption_rate'].mean(),
    'pm25': state_df['pm25'].mean(),
    'median_income': state_df['median_income'].mean(),
    'free_lunch_pct': state_df['free_lunch_pct'].mean()
}

labels_small = ['ESB Adoption Rate (%)', 'PM2.5 (µg/m³)']
bakersfield_small = [round(adoption, 2), round(pm25, 2)]
state_small = [round(state_mean['esb_adoption_rate'], 2), round(state_mean['pm25'], 2)]

fig_bar_small = go.Figure(data=[
    go.Bar(name='Bakersfield', x=labels_small, y=bakersfield_small, marker_color='indianred', text=bakersfield_small, textposition='outside'),
    go.Bar(name='California Avg', x=labels_small, y=state_small, marker_color='lightblue', text=state_small, textposition='outside')
])

fig_bar_small.update_layout(
    barmode='group',
    title="Bakersfield vs California – ESB Adoption & PM2.5",
    yaxis_title="Value",
    yaxis=dict(range=[0, 15]),
    template="plotly_white",
    height=380
)
fig_bar_small.show()


In [10]:
labels_income = ['Median Household Income ($)']
bakersfield_income = [round(income, 0)]
state_income = [round(state_mean['median_income'], 0)]

fig_bar_income = go.Figure(data=[
    go.Bar(name='Bakersfield', x=labels_income, y=bakersfield_income,
           marker_color='indianred',
           text=[f"${v/1000:.1f}k" for v in bakersfield_income], textposition='outside'),
    go.Bar(name='California Avg', x=labels_income, y=state_income,
           marker_color='lightblue',
           text=[f"${v/1000:.1f}k" for v in state_income], textposition='outside')
])

fig_bar_income.update_layout(
    barmode='group',
    title="Bakersfield vs California – Median Income Comparison",
    yaxis_title="Income ($)",
    yaxis=dict(range=[0, 100000]),
    template="plotly_white",
    height=350
)
fig_bar_income.show()


In [11]:
labels_free = ['Students Eligible for Free/Reduced Lunch (%)']

bakersfield_free = [round(free_lunch, 2)]
state_free = [round(state_mean['free_lunch_pct'], 2)]

fig_bar_free = go.Figure(data=[
    go.Bar(name='Bakersfield', x=labels_free, y=bakersfield_free,
           marker_color='indianred',
           text=[f"{v:.2f}%" for v in bakersfield_free],
           textposition='outside'),
    go.Bar(name='California Avg', x=labels_free, y=state_free,
           marker_color='lightblue',
           text=[f"{v:.2f}%" for v in state_free],
           textposition='outside')
])

fig_bar_free.update_layout(
    barmode='group',
    title="🍽️ Bakersfield vs California – Student Economic Vulnerability",
    yaxis_title="% of Students Eligible for Free/Reduced Lunch",
    yaxis=dict(range=[0, 100]),  # percentage scale
    template="plotly_white",
    height=380,
)

fig_bar_free.show()

In [12]:
ca_df = df[df['state'].str.contains('California', case=False, na=False)]

fig_scatter = go.Figure()

# Other districts (blue)
fig_scatter.add_trace(go.Scatter(
    x=ca_df['pm25'],
    y=ca_df['esb_adoption_rate'],
    mode='markers',
    marker=dict(size=7, color='skyblue', opacity=0.5),
    name='Other Districts',
    hovertemplate="<b>%{text}</b><br>PM2.5: %{x:.1f}<br>Adoption: %{y:.1f}%<extra></extra>",
    text=ca_df['district']
))

# Bakersfield districts (red)
fig_scatter.add_trace(go.Scatter(
    x=bakersfield_df['pm25'],
    y=bakersfield_df['esb_adoption_rate'],
    mode='markers',
    marker=dict(size=12, color='red', line=dict(color='black', width=1)),
    name='Bakersfield Districts',
    hovertemplate="<b>%{text}</b><br>PM2.5: %{x:.1f}<br>Adoption: %{y:.1f}%<extra></extra>",
    text=bakersfield_df['district']
))

fig_scatter.update_layout(
    title="PM2.5 vs ESB Adoption Rate – California Districts",
    xaxis_title="Average PM2.5 (µg/m³)",
    yaxis_title="ESB Adoption Rate (%)",
    template="plotly_white",
    height=450
)
fig_scatter.show()


In [13]:
# Sort Bakersfield districts by Free Lunch %
trend_df = bakersfield_df[['district', 'free_lunch_pct']].dropna().sort_values(by='free_lunch_pct')

fig_trend = px.line(
    trend_df,
    x='district',
    y='free_lunch_pct',
    markers=True,
    title="Student Economic Vulnerability by District – Bakersfield, CA",
    labels={'district': 'District', 'free_lunch_pct': '% Eligible for Free/Reduced Lunch'},
    template="plotly_white"
)

fig_trend.update_layout(xaxis_tickangle=-45, height=450)
fig_trend.show()